In [22]:
import tensorflow as tf
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Conv2D, DepthwiseConv2D, BatchNormalization, Activation, GlobalAvgPool2D, Reshape, Dropout
from tensorflow.keras import backend as K

In [16]:
def _Convblocks(inputs, filters, strides=(1, 1), kernal=(3, 3)):
    x = Conv2D(filters, 
               strides = strides, 
               kernel_size = kernal, 
               padding = 'same',
               use_bias = False,
               name='conv1')(inputs)
    x = BatchNormalization(name='conv1_bn')(x)
    
    return Activation(relu6, name='conv1_relu')(x)

In [17]:
def _Depwidth_conv_blocks(inputs, filters, depth_multiplier=1, strides=(1, 1), block_id=1):
    x = DepthwiseConv2D((3, 3), 
                        padding = 'same',
                        strides = strides,
                        depth_multiplier = depth_multiplier,
                        use_bias = False,
                        name = 'conv_dw_%d_'%block_id)(inputs)
    x = BatchNormalization(name = 'conv_bn_%d_'%block_id)(x)
    x = Activation(relu6, name = 'conv_relu_%d_'%block_id)(x)
    
    x = Conv2D(filters, (1, 1), 
               padding = 'same',
               use_bias = False,
               strides = strides,
               name = 'conv_pw_%d_'%block_id)(x)
    x = BatchNormalization(name = 'conv_pw_bn_%d_'%block_id)(x)
    
    return Activation(relu6, name = 'conv_pw_relu_%d_'%block_id)(x)

In [18]:
def relu6(x):
    return K.relu(x, max_value=6)

In [28]:
def Mobilenet(inputshape=[224, 224, 3], depth_multiplier=1,  dropout=1e-3,
              classes=1000):
    
    input_image = Input(shape=inputshape)
    # 224,224,3 -> 112,112,32
    x = _Convblocks(input_image, filters=32, strides=(2, 2))
    
    # 112,112,32 -> 112,112,64
    x = _Depwidth_conv_blocks(x, filters=64, depth_multiplier=depth_multiplier, block_id=1)
    
    # 112,112,64 -> 56,56,128
    x = _Depwidth_conv_blocks(x, filters=128, depth_multiplier=depth_multiplier, strides=(2, 2),block_id=2)
    
    # 56,56,128 -> 56,56,128
    x = _Depwidth_conv_blocks(x, filters=128, depth_multiplier=depth_multiplier, block_id=3)
    
    # 56,56,128 -> 28,28,256
    x = _Depwidth_conv_blocks(x, filters=256, depth_multiplier=depth_multiplier, strides=(2, 2),block_id=4)
    
    # 28,28,256 -> 28,28,256
    x = _Depwidth_conv_blocks(x, filters=256, depth_multiplier=depth_multiplier, block_id=5)
    
    # 28,28,256 -> 14,14,512
    x = _Depwidth_conv_blocks(x, filters=512, depth_multiplier=depth_multiplier, strides=(2, 2),block_id=6)
    
    # 14,14,512 -> 14,14,512 
    x = _Depwidth_conv_blocks(x, filters=512, depth_multiplier=depth_multiplier, block_id=7)
    x = _Depwidth_conv_blocks(x, filters=512, depth_multiplier=depth_multiplier, block_id=8)
    x = _Depwidth_conv_blocks(x, filters=512, depth_multiplier=depth_multiplier, block_id=9)
    x = _Depwidth_conv_blocks(x, filters=512, depth_multiplier=depth_multiplier, block_id=10)
    x = _Depwidth_conv_blocks(x, filters=512, depth_multiplier=depth_multiplier, block_id=11)
    
    # 14,14,512 -> 7,7,1024
    x = _Depwidth_conv_blocks(x, filters=1024, depth_multiplier=depth_multiplier, strides=(2, 2),block_id=12)
    
    # 7,7,1024 -> 7,7,1024
    x = _Depwidth_conv_blocks(x, filters=1024, depth_multiplier=depth_multiplier, block_id=13)
    
    # averagepooling + FC
    x = GlobalAvgPool2D()(x)
    x = Reshape((1, 1, 1024), name='reshape_1')(x)
    x = Dropout(dropout, name='drop_1')(x)
    
    # classfication
    x = Conv2D(classes, (1, 1),padding='same', name='conv_preds')(x)
    x = Activation('softmax', name='act_softmax')(x)
    x = Reshape((classes,), name='reshape_2')(x)
    
    inputs = input_image
    model = Model(inputs, x, name='mobilenet_1_0_224_tf')
    
    return model

In [29]:
model = Mobilenet()

In [30]:
model.summary()

Model: "mobilenet_1_0_224_tf"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (Activation)      (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1_ (DepthwiseConv2D) (None, 112, 112, 32)      288       
_________________________________________________________________
conv_bn_1_ (BatchNormalizati (None, 112, 112, 32)      128       
_________________________________________________________________
conv_relu_1_ (Activation)    (None, 112, 112, 